# Exploring Data from multiple sources

In [6]:
import csv
import pandas as pd
import os

### San Bernardino County, CA data points on drought

In [10]:
path = os.getcwd()
print(path)

droughtData = pd.read_csv(path + '/src/SB_DroughtConditions.csv')
print(droughtData)

/Users/aimeeoh/Desktop/Documents/ColumbiaMS/MLC/ML-Climate
       MapDate  FIPS                 County State   None      D0     D1  \
0     20250401  6071  San Bernardino County    CA   0.00  100.00  100.0   
1     20250325  6071  San Bernardino County    CA   0.00  100.00  100.0   
2     20250318  6071  San Bernardino County    CA   0.00  100.00  100.0   
3     20250311  6071  San Bernardino County    CA   0.00  100.00  100.0   
4     20250304  6071  San Bernardino County    CA   0.00  100.00  100.0   
...        ...   ...                    ...   ...    ...     ...    ...   
1313  20000201  6071  San Bernardino County    CA   0.00  100.00    0.0   
1314  20000125  6071  San Bernardino County    CA   0.00  100.00    0.0   
1315  20000118  6071  San Bernardino County    CA   0.00  100.00    0.0   
1316  20000111  6071  San Bernardino County    CA   0.00  100.00    0.0   
1317  20000104  6071  San Bernardino County    CA  85.97   14.03    0.0   

         D2     D3   D4  ValidStart    V